In [1]:
import normals_lib
import pointareas as pareas
import kaolin as kal
import torch
import numpy as np
import meshplot as mp
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import open3d as o3d

%matplotlib tk
#mesh = kal.rep.TriangleMesh.from_obj("model.obj") # Modelo de una esfera
#mesh = kal.rep.TriangleMesh.from_obj("brain.obj")
#mesh = kal.rep.TriangleMesh.from_obj("bunny.obj")
#mesh = kal.rep.TriangleMesh.from_obj("cow.obj")
mesh = kal.rep.TriangleMesh.from_obj("horse.obj")
#mesh = kal.rep.TriangleMesh.from_obj("maxplanck.obj")

In [90]:
viewpos = torch.zeros(mesh.vertices.shape)
with open("/home/danniccs/rtsc-1.6/viewpos", "r") as myfile:
    l = myfile.readline()
    for i in range(0, viewpos.shape[0]):
        viewpos[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])
viewpos = viewpos.cuda()

In [89]:
viewcoords = torch.load('view_coordinates.pt')

In [3]:
normals = normals_lib.compute_simple_vertex_normals(mesh)
pointareas, cornerareas = pareas.compute_pointareas(mesh)
normals_lib.visualize_normals(mesh, normals.cpu())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.991888…

In [4]:
# Calculate principal curvatures and curvature directions
import curvature
import importlib

k1, k2, e1, e2 = curvature.compute_curvatures(mesh, method='cholesky', normals=normals, pointareas=pointareas,
                                              cornerareas=cornerareas)

In [ ]:
altk1, altk2, _, _ = curvature.compute_curvatures(mesh, method="cholesky", normals=normals,
                                                  pointareas=pointareas, cornerareas=cornerareas)

In [5]:
import dcurv

dc = dcurv.compute_dcurvs(mesh, method='cholesky', curvs=(k1,k2,e1,e2), normals=normals, pointareas=pointareas,
                          cornerareas=cornerareas)

In [21]:
importlib.reload(dcurv)

<module 'dcurv' from '/home/danniccs/rtsc_to_torch/dcurv.py'>

In [91]:
DwKr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
kr = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
for i in range(0, 12):
    DwKr[i], kr[i] = dcurv.compute_DwKr(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc, view_coords=viewpos)

In [92]:
for i in range(0, 12):
    kupper = int(DwKr[i].shape[0] * 0.95)
    klower = int(DwKr[i].shape[0] * 0.05)
    dwupper, _ = torch.kthvalue(DwKr[i], kupper)
    dwlower, _ = torch.kthvalue(DwKr[i], klower)
    DwKr[i][DwKr[i] > dwupper] = dwupper
    DwKr[i][DwKr[i] < dwlower] = dwlower
    kupper = int(kr[i].shape[0] * 0.95)
    klower = int(kr[i].shape[0] * 0.05)
    krupper, _ = torch.kthvalue(kr[i], kupper)
    krlower, _ = torch.kthvalue(kr[i], klower)
    kr[i][kr[i] > krupper] = krupper
    kr[i][kr[i] < krlower] = krlower

In [22]:
DwKr_alt = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
kr_alt = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
for i in range(0, 12):
    DwKr_alt[i], kr_alt[i] = dcurv.compute_DwKr_alt(mesh, normals=normals, curvs=(k1,k2,e1,e2), dcurv=dc,
                                                    view_coords=viewpos)
    kupper = int(DwKr_alt[i].shape[0] * 0.95)
    klower = int(DwKr_alt[i].shape[0] * 0.05)
    dwupper, _ = torch.kthvalue(DwKr_alt[i], kupper)
    dwlower, _ = torch.kthvalue(DwKr_alt[i], klower)
    DwKr_alt[i][DwKr_alt[i] > dwupper] = dwupper
    DwKr_alt[i][DwKr_alt[i] < dwlower] = dwlower
    kupper = int(kr_alt[i].shape[0] * 0.95)
    klower = int(kr_alt[i].shape[0] * 0.05)
    krupper, _ = torch.kthvalue(kr_alt[i], kupper)
    krlower, _ = torch.kthvalue(kr_alt[i], klower)
    kr_alt[i][kr_alt[i] > krupper] = krupper
    kr_alt[i][kr_alt[i] < krlower] = krlower

In [6]:
kupper = int(k1.shape[0] * 0.95)
klower = int(k1.shape[0] * 0.05)
dwupper, _ = torch.kthvalue(k1, kupper)
dwlower, _ = torch.kthvalue(k1, klower)
k1[k1 > dwupper] = dwupper
k1[k1 < dwlower] = dwlower
kupper = int(k2.shape[0] * 0.95)
klower = int(k2.shape[0] * 0.05)
krupper, _ = torch.kthvalue(k2, kupper)
krlower, _ = torch.kthvalue(k2, klower)
k2[k2 > krupper] = krupper
k2[k2 < krlower] = krlower

In [7]:
kupper = int(altk1.shape[0] * 0.95)
klower = int(altk1.shape[0] * 0.05)
dwupper, _ = torch.kthvalue(altk1, kupper)
dwlower, _ = torch.kthvalue(altk1, klower)
altk1[altk1 > dwupper] = dwupper
altk1[altk1 < dwlower] = dwlower
kupper = int(altk2.shape[0] * 0.95)
klower = int(altk2.shape[0] * 0.05)
krupper, _ = torch.kthvalue(altk2, kupper)
krlower, _ = torch.kthvalue(altk2, klower)
altk2[altk2 > krupper] = krupper
altk2[altk2 < krlower] = krlower

In [8]:
trik1 = torch.zeros(k1.shape).cuda()
trik2 = torch.zeros(k2.shape).cuda()
trie1 = torch.zeros(e1.shape).cuda()
trie2 = torch.zeros(e2.shape).cuda()

with open("/home/danniccs/rtsc-1.6/k1curv", "r") as myfile:
    for i in range(0, k1.shape[0]):
        l = myfile.readline()
        trik1[i] = float(l)

with open("/home/danniccs/rtsc-1.6/k2curv", "r") as myfile:
    for i in range(0, k2.shape[0]):
        l = myfile.readline()
        trik2[i] = float(l)

with open("/home/danniccs/rtsc-1.6/pdir1", "r") as myfile:
    for i in range(0, e1.shape[0]):
        l = myfile.readline()
        trie1[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])

with open("/home/danniccs/rtsc-1.6/pdir2", "r") as myfile:
    for i in range(0, e2.shape[0]):
        l = myfile.readline()
        trie2[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])

In [93]:
tridc = torch.zeros(dc.shape)
dwtri = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
krtri = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
with open("/home/danniccs/rtsc-1.6/dcurv", "r") as myfile:
    for i in range(0, dc.shape[0]):
        l = myfile.readline()
        tridc[i] = torch.tensor([float(n) for n in l[1:-2].split(',')])

for i in range(0, 12):
    dwtri[i], krtri[i] = dcurv.compute_DwKr(mesh, normals=normals, curvs=(trik1,trik2,trie1,trie2),
                                            dcurv=tridc.cuda(), view_coords=viewpos)

In [94]:
for i in range(0, 12):
    kupper = int(dwtri[i].shape[0] * 0.95)
    klower = int(dwtri[i].shape[0] * 0.05)
    dwupper, _ = torch.kthvalue(dwtri[i], kupper)
    dwlower, _ = torch.kthvalue(dwtri[i], klower)
    dwtri[i][dwtri[i] > dwupper] = dwupper
    dwtri[i][dwtri[i] < dwlower] = dwlower
    kupper = int(krtri[i].shape[0] * 0.95)
    klower = int(krtri[i].shape[0] * 0.05)
    dwupper, _ = torch.kthvalue(krtri[i], kupper)
    dwlower, _ = torch.kthvalue(krtri[i], klower)
    krtri[i][krtri[i] > dwupper] = dwupper
    krtri[i][krtri[i] < dwlower] = dwlower

In [22]:
import shared_algs
ndotv = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
krf = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest_num = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest_den = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))
sctest = torch.zeros(12, mesh.vertices.shape[0]).to(device=torch.device('cuda'))

tridc = tridc.cuda()
for i in range(0, 12):
    ndotv[i], krf[i], _, sctest_num[i], sctest_den[i] = shared_algs.compute_perview(mesh, normals=normals,
                                                                                    curvs=(trik1,trik2,trie1,trie2),
                                                                                    dcurv=tridc,
                                                                                    view_coords=viewpos)
    sctest[i] = sctest_num[i] / sctest_den[i]

In [23]:
for i in range(0, 12):
    #sctest_den[i][sctest_den[i] == 0] = 0.01
    kupper = int(sctest[i].shape[0] * 0.95)
    klower = int(sctest[i].shape[0] * 0.05)
    scupper, _ = torch.kthvalue(sctest[i], kupper)
    sclower, _ = torch.kthvalue(sctest[i], klower)
    sctest[i][sctest[i] > scupper] = scupper
    sctest[i][sctest[i] < sclower] = sclower
    
    kupper = int(krf[i].shape[0] * 0.95)
    klower = int(krf[i].shape[0] * 0.05)
    scupper, _ = torch.kthvalue(krf[i], kupper)
    sclower, _ = torch.kthvalue(krf[i], klower)
    krf[i][krf[i] > scupper] = scupper
    krf[i][krf[i] < sclower] = sclower

In [24]:
trisctest = torch.zeros(sctest[0].shape)

with open("/home/danniccs/rtsc-1.6/sctest", "r") as myfile:
    for i in range(0, trisctest.shape[0]):
        l = myfile.readline()
        trisctest[i] = float(l)

kupper = int(trisctest.shape[0] * 0.95)
klower = int(trisctest.shape[0] * 0.05)
scupper, _ = torch.kthvalue(trisctest, kupper)
sclower, _ = torch.kthvalue(trisctest, klower)
trisctest[trisctest > scupper] = scupper
trisctest[trisctest < sclower] = sclower

In [10]:
kupper = int(trik1.shape[0] * 0.95)
klower = int(trik1.shape[0] * 0.05)
dwupper, _ = torch.kthvalue(trik1, kupper)
dwlower, _ = torch.kthvalue(trik1, klower)
trik1[trik1 > dwupper] = dwupper
trik1[trik1 < dwlower] = dwlower
kupper = int(trik2.shape[0] * 0.95)
klower = int(trik2.shape[0] * 0.05)
krupper, _ = torch.kthvalue(trik2, kupper)
krlower, _ = torch.kthvalue(trik2, klower)
trik2[trik2 > krupper] = krupper
trik2[trik2 < krlower] = krlower

In [57]:
import matplotlib.colors as mlcol
import matplotlib.cm as cm

nmalizer = mlcol.Normalize()
scmap = cm.ScalarMappable(norm=nmalizer, cmap=plt.get_cmap("jet"))
mesh_rgba = scmap.to_rgba(DwKr[0].cpu().numpy())
mesh_rgba = np.delete(mesh_rgba, 3, 1)

In [8]:
mp.jupyter()
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k1.cpu().numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=altk1.cpu().numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [9]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=k2.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=altk2.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [86]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=e1.to(device=torch.device('cpu')).numpy(),
               s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=trie1.to(device=torch.device('cpu')).numpy(),
               s=[1,2,1], data=d)

In [86]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=dwtri[1].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [128]:
d = mp.plot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, plot=mp.Viewer({}), return_plot=True)
d._orbit.autoRotate = True
d._orbit.autoRotateSpeed = 5
d._orbit.rotateSpeed = 5
d.update_object()
d._renderer.controls = [d._orbit]
display(d._renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-4.991888…

In [98]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=kr[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=krtri[1].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [27]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=DwKr[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [28]:
d = mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=sctest[0].to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,0])
mp.subplot(mesh.vertices.numpy(), mesh.faces.numpy(), c=trisctest.to(device=torch.device('cpu')).numpy(),
            shading={'colormap': "jet_r"}, s=[1,2,1], data=d)

In [167]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de DwKr\n')

hplot = sns.histplot(ax=axes[0], data=DwKr[0].cpu().numpy())
hplot.set(xlabel='Valor de DwKr')
axes[0].set_title('pyrtsc')

hplot = sns.histplot(ax=axes[1], data=dwtri[0].cpu().numpy())
hplot.set(xlabel='Valor de DwKr')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [62]:
sns.set_theme(style="whitegrid")

df = pd.DataFrame(np.array([DwKr[0].cpu().numpy(), dwtri[0].cpu().numpy()]).transpose(), columns=['pyrtsc', 'rtsc'])
bplot = sns.boxplot(data=df)
bplot.set(ylabel='Valor de DwKr')
bplot.set_title('Comparación de distribuciones de DwKr')
#plt.show()

Text(0.5, 1.0, 'Comparación de distribuciones de DwKr')

In [ ]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de DwKr y sctest\n')
sns.set_theme(style="white")

hplot = sns.histplot(ax=axes[0], data=DwKr[0].cpu().numpy())
hplot.set(xlabel='Valor de DwKr')

hplot = sns.histplot(ax=axes[1], data=sctest[0].cpu().numpy())
hplot.set(xlabel='Valor de sctest')
#plt.show()

In [11]:
fig, axes = plt.subplots(1, 2, sharey=False)
fig.suptitle('Comparación de distribuciones de DwKr y sctest\n')
sns.set_theme(style="whitegrid")

bplot = sns.boxplot(ax=axes[0], y=DwKr[0].cpu().numpy())
bplot.set(ylabel='Valor de DwKr')

bplot = sns.boxplot(ax=axes[1], y=sctest[0].cpu().numpy())
bplot.set(ylabel='Valor de sctest')
#plt.show()

[Text(0, 0.5, 'Valor de sctest')]

In [115]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de curvaturas radiales\n')
sns.set_theme(style="white")

hplot = sns.histplot(ax=axes[0], data=kr[0].cpu().numpy())
hplot.set(xlabel='Valor de kr')
axes[0].set_title('pyrtsc')

hplot = sns.histplot(ax=axes[1], data=krtri[0].cpu().numpy())
hplot.set(xlabel='Valor de kr')
axes[1].set_title('rtsc')
#plt.show()

Text(0.5, 1.0, 'rtsc')

In [66]:
sns.set_theme(style="whitegrid")

df = pd.DataFrame(np.array([kr[0].cpu().numpy(), krtri[0].cpu().numpy()]).transpose(), columns=['pyrtsc', 'rtsc'])
bplot = sns.boxplot(data=df)
bplot.set(ylabel='Valor de kr')
bplot.set_title('Comparación de distribuciones de kr')
#plt.show()

Text(0.5, 1.0, 'Comparación de distribuciones de kr')

In [12]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de curvatura principal k1\n')

hplot = sns.histplot(ax=axes[0], data=k1.cpu().numpy())
hplot.set(xlabel='Valor de k1')
axes[0].set_title('lstsq')

hplot = sns.histplot(ax=axes[1], data=altk1.cpu().numpy())
hplot.set(xlabel='Valor de k1')
axes[1].set_title('Cholesky')
#plt.show()

Text(0.5, 1.0, 'Cholesky')

In [7]:
fig, axes = plt.subplots(1, 2, sharey=True)
fig.suptitle('Comparación de distribuciones de curvatura principal k2\n')

hplot = sns.histplot(ax=axes[0], data=k2.cpu().numpy())
hplot.set(xlabel='Valor de k2')
axes[0].set_title('lstsq')

hplot = sns.histplot(ax=axes[1], data=altk2.cpu().numpy())
hplot.set(xlabel='Valor de k2')
axes[1].set_title('Cholesky')
#plt.show()

Text(0.5, 1.0, 'Cholesky')

In [49]:
fig, axes = plt.subplots(1, 2)

hplot = sns.histplot(data=e1.cpu().numpy(), ax=axes[0])

hplot = sns.histplot(data=trie1.cpu().numpy(), ax=axes[1])

In [18]:
fig, axes = plt.subplots(1, 2, subplot_kw=dict(projection="polar"))

theta = np.arccos(e1[:,2].cpu().numpy())
df = pd.DataFrame(theta, columns=['theta'])
hplot = sns.histplot(data=df, ax=axes[0])
axes[0].set_rlabel_position(270)
axes[0].set(ylabel="")

tritheta = np.arccos(trie1[:,2].cpu().numpy())
tridf = pd.DataFrame(tritheta, columns=['theta'])
hplot = sns.histplot(data=tridf, ax=axes[1])
axes[1].set_rlabel_position(270)
axes[1].set(ylabel="")

[Text(0, 0.5, '')]

In [19]:
fig, axes = plt.subplots(1, 2, subplot_kw=dict(projection="polar"))

phi = np.arctan2(e1[:,1].cpu().numpy(), e1[:,0].cpu().numpy())
df = pd.DataFrame(phi, columns=['phi'])
hplot = sns.histplot(data=df, ax=axes[0])
axes[0].set_rlabel_position(270)
axes[0].set(ylabel="")

triphi = np.arctan2(trie1[:,1].cpu().numpy(), trie1[:,0].cpu().numpy())
tridf = pd.DataFrame(triphi, columns=['phi'])
hplot = sns.histplot(data=tridf, ax=axes[1])
axes[1].set_rlabel_position(270)
axes[1].set(ylabel="")

[Text(0, 0.5, '')]

In [30]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

ax = plt.subplot()
ax.set_axis_off()
dset = DwKr[0].cpu().numpy()
im = ax.imshow(np.expand_dims(np.linspace(np.floor(dset.min()), np.ceil(dset.max()), 50), 1),
               cmap=plt.get_cmap("jet_r"))

# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

plt.colorbar(im, cax=cax)

plt.show()

In [10]:
import pickle

with open("./curvtimes", "rb") as myfile:
    speedDict = pickle.load(myfile)

with open("./curvtimes_chol", "rb") as myfile:
    speedDictChol = pickle.load(myfile)

{'cow': [16.080965301, 16.00931484099999, 16.41002288599998, 15.940427124000053, 16.09649451899986, 16.031626481000785, 16.00473958099974, 15.965231146000406, 16.308280651000132, 16.043601514999864], 'brain': [100.92670668099998, 101.28925802399999, 100.36651987599998, 100.33858868900006, 101.04065844299976, 101.3302612940006, 101.28351850800027, 100.690533555, 101.28376479500002, 101.06968883900026], 'bunny': [186.666959565, 186.50333699200007, 185.30018377400006, 185.32369976400014, 186.90193626400014, 186.99818353400042, 185.48347873399962, 186.36609706699983, 186.21869161799987, 185.4308589049997], 'horse': [256.89135172699997, 256.7492589249998, 257.09431783800005, 258.6008604660001, 258.65404698400016, 259.2155534169997, 257.3367465559995, 259.56159619400023, 257.59126116100015, 257.36840205599947], 'maxplanck': [259.607598521, 260.04117358300005, 260.09948306399974, 261.72596415499993, 262.36485469599984, 262.0512081630004, 260.2602813409994, 263.2174520569997, 261.0510267529998

In [35]:
avgTimes = []
for k,v in speedDict.items():
    avgTimes.append(np.mean(v))

print(avgTimes)
    
avgTimesChol = []
for k,v in speedDictChol.items():
    avgTimesChol.append(np.mean(v))

print(avgTimesChol)

face_nums = []
with open("/home/danniccs/rtsc_to_torch/face_numbers", "r") as myfile:
    for i in range(0, 5):
        l = myfile.readline()
        face_nums.append(float(l))

avgTimesAll = np.array([avgTimes, avgTimesChol], ndmin=2)
avgTimesAll = avgTimesAll.transpose()
#df = pd.DataFrame(avgTimesAll, columns=['Cow', 'Brain', 'Bunny', 'Horse', 'Max Planck'])
df = pd.DataFrame(avgTimesAll, columns=['lstsq', 'cholesky'], index=face_nums)
linplot = sns.lineplot(data=df, dashes=[(1,2),(1,2)], markers=True)
linplot.set(xlabel='Número de caras', ylabel='Tiempo de cálculo de curvaturas (segundos)')
linplot.set_title('Tiempos de cálculo de curvaturas')

df

[16.08907040450008, 100.9619498704001, 186.11934262169999, 257.90633953239995, 261.05572298279986]
[14.212133667100048, 89.27507912809997, 163.70395577739993, 226.53156685459993, 229.52960536780006]


,lstsq,cholesky
5804.0,16.089070,14.212134
36752.0,100.961950,89.275079
69451.0,186.119343,163.703956
96964.0,257.906340,226.531567
98260.0,261.055723,229.529605
